In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
5,23andMe research finds possible link between b...,https://www.statnews.com/2020/09/14/23andme-st...,"Tue, 15 Sep 2020 01:52:30 GMT",2020-09-15 01:52:30+00:00
4,Covid-19 News: Live Updates The New York Times,https://www.nytimes.com/2020/09/14/world/covid...,"Tue, 15 Sep 2020 01:36:00 GMT",2020-09-15 01:36:00+00:00
7,"College campuses are COVID-19 hot spots, state...",https://www.detroitnews.com/story/news/educati...,"Tue, 15 Sep 2020 00:35:29 GMT",2020-09-15 00:35:29+00:00
0,Coronavirus (COVID-19) Update: Daily Roundup S...,https://www.fda.gov/news-events/press-announce...,"Mon, 14 Sep 2020 21:00:00 GMT",2020-09-14 21:00:00+00:00
9,UW announces COVID-19 testing program for stud...,https://www.washington.edu/news/2020/09/14/uw-...,"Mon, 14 Sep 2020 17:59:53 GMT",2020-09-14 17:59:53+00:00
6,California Coronavirus Updates: CDC Study Show...,https://www.capradio.org/articles/2020/09/14/c...,"Mon, 14 Sep 2020 16:52:30 GMT",2020-09-14 16:52:30+00:00
8,Governor Cuomo Announces 38th Straight Day Wit...,https://www.governor.ny.gov/news/governor-cuom...,"Mon, 14 Sep 2020 16:22:10 GMT",2020-09-14 16:22:10+00:00
3,14% of U.S. adults say they have tested positi...,https://www.pewresearch.org/fact-tank/2020/09/...,"Mon, 14 Sep 2020 16:15:43 GMT",2020-09-14 16:15:43+00:00
1,Substance use disorders linked to COVID-19 sus...,https://www.nih.gov/news-events/news-releases/...,"Mon, 14 Sep 2020 14:06:03 GMT",2020-09-14 14:06:03+00:00
2,COVID-19 Daily Update 9-14-2020 West Virginia...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Mon, 14 Sep 2020 14:05:41 GMT",2020-09-14 14:05:41+00:00
